In [4]:
#Install nltk
#pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [20]:
#IMPORT THE REQUIRED LIBRARIES

import nltk
import sqlite3
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Joakim\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [25]:
sia = SentimentIntensityAnalyzer()

# Connect to the SQLite database
conn = sqlite3.connect("instagram_data.db")
cursor = conn.cursor()

# Create the sentiment_analysis table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS sentiment_analysis (
    id TEXT PRIMARY KEY,
    negative REAL,
    neutral REAL,
    positive REAL,
    compound REAL,
    FOREIGN KEY (id) REFERENCES instagram_posts(id) ON DELETE CASCADE
)
""")

# Query to fetch all posts from instagram_posts
cursor.execute("SELECT id, caption FROM instagram_posts")
rows = cursor.fetchall()

# Prepare data for insertion
sentiment_data = []
for row in rows:
    post_id, caption = row
    sentiment = sia.polarity_scores(caption)

    # Extract scores
    negative = sentiment['neg']
    neutral = sentiment['neu']
    positive = sentiment['pos']
    compound = sentiment['compound']

    # Append to list
    sentiment_data.append((post_id, negative, neutral, positive, compound))

# Insert sentiment analysis results into the new table
cursor.executemany("""
INSERT OR REPLACE INTO sentiment_analysis 
(id, negative, neutral, positive, compound) 
VALUES (?, ?, ?, ?, ?)
""", sentiment_data)

# Commit and close
conn.commit()
conn.close()

print("Sentiment analysis data stored successfully!")

Sentiment analysis data stored successfully!


In [26]:
# PRINTING ALL THE POSTS AND THEIR RESPECTIVE SCORES
# Connect to the SQLite database
conn = sqlite3.connect("instagram_data.db")
cursor = conn.cursor()


cursor.execute("SELECT * FROM sentiment_analysis")
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=headers)

print(df)
# Close the connection
conn.close()

                  id  negative  neutral  positive  compound
0  17920309653044896       0.0    0.734     0.266    0.4404
1  18034496246531367       0.0    0.000     0.000    0.0000
2  18090093601497188       0.0    0.000     0.000    0.0000
3  17861666277364573       0.0    0.691     0.309    0.9245


In [31]:
post_id = "17920309653044896"  # Replace with the actual post ID

conn = sqlite3.connect("instagram_data.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM sentiment_analysis WHERE id = ?", (post_id,))
result = cursor.fetchone()

if result:
    print(f"Post ID: {result[0]}, Negative: {result[1]}, Neutral: {result[2]}, Positive: {result[3]}, Compound: {result[4]}")
else:
    print("No sentiment data found for this post.")

# Close connection
conn.close()

Post ID: 17920309653044896, Negative: 0.0, Neutral: 0.734, Positive: 0.266, Compound: 0.4404
